# Libraries

In [11]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import nltk.data
import spacy

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ferencipeter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Import Data

In [4]:
data = pd.read_csv('data/nyt.csv')

In [5]:
data.head()

,Date,Headline,Snippet
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi..."
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...


In [7]:
data.shape

(184, 3)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 3 columns):
Date        184 non-null object
Headline    184 non-null object
Snippet     184 non-null object
dtypes: object(3)
memory usage: 4.4+ KB


# Text Cleaning

In [42]:
def text_cleanup(text):
    text = text.replace('RT ', '')
    text = text.replace('\n', '')
    clean_text = []
    for word in text.split(' '):
        if word.startswith('http://'):
            word = ''
        if word.startswith('https://'):
            word = ''
        if word.startswith('.@'):
            word = word[1:]
        if word.endswith('.'):
            word = word[:-1]
        clean_text.append(word)
    return ' '.join(clean_text)


for i in data.index.tolist():
    data.at[i,'Headline_Cleaned'] = text_cleanup(data['Headline'][i])

In [50]:
for i in data.index.tolist():
    data.at[i,'Snippet_Cleaned'] = text_cleanup(data['Snippet'][i])

In [51]:
data.head()

,Date,Headline,Snippet,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment,Headline_polarity,Snippet_polarity,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Snippet_Cleaned
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,-0.4767,0,Donald Trump Opens New Line of Attack on Hilla...,"Donald, Trump, Opens, New, Line, Attack, Hilla...","of, on, Her","Donald, Trump, Opens, New, Line, of, Attack, o...","In an interview, Mr Trump said he was reconsid..."
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'com...",neu,0,0.5859,Hillary Clinton and Democrats Raised $154 Mill...,"Hillary, Clinton, Democrats, Raised, $, 154, M...","and, in","Hillary, Clinton, and, Democrats, Raised, $, 1...",Her campaign and the party have a combined cas...
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...,"[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'com...",neu,"[{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'com...",neu,0.3182,0.2732,New Hillary Clinton Ads Look to Expand Lead Wi...,"New, Hillary, Clinton, Ads, Look, Expand, Lead...","to, With","New, Hillary, Clinton, Ads, Look, to, Expand, ...",The Clinton team has made Hispanic outreach a ...
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?",It’s time for Republican leaders to exercise a...
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...,"[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu,"[{'neg': 0.116, 'neu': 0.884, 'pos': 0.0, 'com...",neu,0.296,-0.4404,Donald Trump Is Seen as Helping Push Asian-Ame...,"Donald, Trump, Seen, Helping, Push, Asian, -, ...","Is, as, Into","Donald, Trump, Is, Seen, as, Helping, Push, As...",Asian-Americans are identifying as Democrats a...


# Stop Words and Tokenization

In [46]:
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS

data['Headline_Keywords'] = ""
data['Headline_Stopwords'] = ""
data['Headline_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Headline_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Headline_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Headline_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

---------
Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage
---------
---------
['Donald', 'Trump', 'Opens', 'New', 'Line', 'Attack', 'Hillary', 'Clinton', ':', 'Marriage']
['of', 'on', 'Her']
---------
---------
Hillary Clinton and Democrats Raised $154 Million in September
---------
---------
['Hillary', 'Clinton', 'Democrats', 'Raised', '$', '154', 'Million', 'September']
['and', 'in']
---------
---------
New Hillary Clinton Ads Look to Expand Lead With Hispanic Voters
---------
---------
['New', 'Hillary', 'Clinton', 'Ads', 'Look', 'Expand', 'Lead', 'Hispanic', 'Voters']
['to', 'With']
---------
---------
How Could Anyone Vote for Trump?
---------
---------
['Vote', 'Trump', '?']
['How', 'Could', 'Anyone', 'for']
---------
---------
Donald Trump Is Seen as Helping Push Asian-Americans Into Democratic Arms
---------
---------
['Donald', 'Trump', 'Seen', 'Helping', 'Push', 'Asian', '-', 'Americans', 'Democratic', 'Arms']
['Is', 'as', 'Into']
---------
---------
D

---------
New York Today: A City of Superheroes
---------
---------
['New', 'York', 'Today', ':', 'City', 'Superheroes']
['A', 'of']
---------
---------
‘Doomocracy’ Puts the Politics of Fear on Display in Brooklyn
---------
---------
['‘', 'Doomocracy', '’', 'Puts', 'Politics', 'Fear', 'Display', 'Brooklyn']
['the', 'of', 'on', 'in']
---------
---------
At Frieze London, Throwing Light on Tense Times
---------
---------
['Frieze', 'London', ',', 'Throwing', 'Light', 'Tense', 'Times']
['At', 'on']
---------
---------
Balance, Trump and Lies
---------
---------
['Balance', ',', 'Trump', 'Lies']
['and']
---------
---------
How This Jobs Report Can Help Clinton, and Trump
---------
---------
['Jobs', 'Report', 'Help', 'Clinton', ',', 'Trump']
['How', 'This', 'Can', 'and']
---------
---------
How Many Times Have Both Parties Nominated New Yorkers for President?
---------
---------
['Times', 'Parties', 'Nominated', 'New', 'Yorkers', 'President', '?']
['How', 'Many', 'Have', 'Both', 'for']
-

---------
Hillary Clinton Builds $150 Million War Chest, Doubling Donald Trump
---------
---------
['Hillary', 'Clinton', 'Builds', '$', '150', 'Million', 'War', 'Chest', ',', 'Doubling', 'Donald', 'Trump']
[]
---------
---------
Specter of Trump Loosens Tongues, if Not Purse Strings, in Silicon Valley
---------
---------
['Specter', 'Trump', 'Loosens', 'Tongues', ',', 'Purse', 'Strings', ',', 'Silicon', 'Valley']
['of', 'if', 'Not', 'in']
---------
---------
Ripples From the ‘How Low Can They Go’ Campaign
---------
---------
['Ripples', '‘', 'Low', '’', 'Campaign']
['From', 'the', 'How', 'Can', 'They', 'Go']
---------
---------
Donald Trump and the G.O.P.: The Party of Lincoln, Reagan and, Perhaps, Extinction
---------
---------
['Donald', 'Trump', 'G.O.P.', ':', 'Party', 'Lincoln', ',', 'Reagan', ',', ',', 'Extinction']
['and', 'the', 'The', 'of', 'and', 'Perhaps']
---------
---------
Officials Fight Donald Trump’s Claims of a Rigged Vote
---------
---------
['Officials', 'Fight', 'D

---------
Hillary Clinton Makes Pitch for Mandate and a Swing-State Sweep
---------
---------
['Hillary', 'Clinton', 'Makes', 'Pitch', 'Mandate', 'Swing', '-', 'State', 'Sweep']
['for', 'and', 'a']
---------
---------
Could Donald Trump Reject the Election Results? Yes Would It Do Any Good? Nope
---------
---------
['Donald', 'Trump', 'Reject', 'Election', 'Results', '?', 'Yes', 'Good', '?', 'Nope']
['Could', 'the', 'Would', 'It', 'Do', 'Any']
---------
---------
Donald Trump’s Defiance Is Seen as ‘Colossal Mistake’ That Threatens U.S Image
---------
---------
['Donald', 'Trump', 'Defiance', 'Seen', '‘', 'Colossal', 'Mistake', '’', 'Threatens', 'U.S', 'Image']
['’s', 'Is', 'as', 'That']
---------
---------
The Hawk on Russia Policy? Hillary Clinton, Not Donald Trump
---------
---------
['Hawk', 'Russia', 'Policy', '?', 'Hillary', 'Clinton', ',', 'Donald', 'Trump']
['The', 'on', 'Not']
---------
---------
In Debate, Hillary Clinton’s Clarion Call for Women Thrills Many
---------
-------

In [52]:
data['Snippet_Keywords'] = ""
data['Snippet_Stopwords'] = ""
data['Snippet_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Snippet_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Snippet_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Snippet_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

---------
In an interview, Mr Trump said he was reconsidering whether he would back Mrs Clinton if she became president
---------
---------
['interview', ',', 'Mr', 'Trump', 'said', 'reconsidering', 'Mrs', 'Clinton', 'president']
['In', 'an', 'he', 'was', 'whether', 'he', 'would', 'back', 'if', 'she', 'became']
---------
---------
Her campaign and the party have a combined cash reserve of $150 million, setting them up for a final spending splurge on television ads and outreach
---------
---------
['campaign', 'party', 'combined', 'cash', 'reserve', '$', '150', 'million', ',', 'setting', 'final', 'spending', 'splurge', 'television', 'ads', 'outreach']
['Her', 'and', 'the', 'have', 'a', 'of', 'them', 'up', 'for', 'a', 'on', 'and']
---------
---------
The Clinton team has made Hispanic outreach a cornerstone of its advertising, and results are showing: In Florida, a poll shows her as having a 35-point lead
---------
---------
['Clinton', 'team', 'Hispanic', 'outreach', 'cornerstone', 'adv

---------
“That Mexican thing” pops up on social media, a new poll shows a tight race in Ohio, and the Republican ticket appears split over how to handle the Russian president
---------
---------
['“', 'Mexican', 'thing', '”', 'pops', 'social', 'media', ',', 'new', 'poll', 'shows', 'tight', 'race', 'Ohio', ',', 'Republican', 'ticket', 'appears', 'split', 'handle', 'Russian', 'president']
['That', 'up', 'on', 'a', 'a', 'in', 'and', 'the', 'over', 'how', 'to', 'the']
---------
---------
Newspaper and magazine editorials denouncing Donald J Trump keep coming, the latest being a rare presidential endorsement by The Atlantic magazine
---------
---------
['Newspaper', 'magazine', 'editorials', 'denouncing', 'Donald', 'J', 'Trump', 'coming', ',', 'latest', 'rare', 'presidential', 'endorsement', 'Atlantic', 'magazine']
['and', 'keep', 'the', 'being', 'a', 'by', 'The']
---------
---------
Reporters for The New York Times fact-checked the statements made by Senator Tim Kaine and Gov Mike Pence d

---------
Shunned by many Republicans over his lewd comments about women, Mr Trump spent most of Friday and Saturday at his headquarters as his advisers pondered his next steps
---------
---------
['Shunned', 'Republicans', 'lewd', 'comments', 'women', ',', 'Mr', 'Trump', 'spent', 'Friday', 'Saturday', 'headquarters', 'advisers', 'pondered', 'steps']
['by', 'many', 'over', 'his', 'about', 'most', 'of', 'and', 'at', 'his', 'as', 'his', 'his', 'next']
---------
---------
Hillary Clinton and Donald J Trump took the stage after Mr Trump held a news conference with women who have accused Bill Clinton of sexual assault over the years
---------
---------
['Hillary', 'Clinton', 'Donald', 'J', 'Trump', 'took', 'stage', 'Mr', 'Trump', 'held', 'news', 'conference', 'women', 'accused', 'Bill', 'Clinton', 'sexual', 'assault', 'years']
['and', 'the', 'after', 'a', 'with', 'who', 'have', 'of', 'over', 'the']
---------
---------
The Republican candidate’s supporters are apparently standing by him desp

---------
Graffiti painted nearby depicted a swastika and the warning, “Nazi Republicans leave town or else.”
---------
---------
['Graffiti', 'painted', 'nearby', 'depicted', 'swastika', 'warning', ',', '“', 'Nazi', 'Republicans', 'leave', 'town', '.', '”']
['a', 'and', 'the', 'or', 'else']
---------
---------
Here’s what you need to know about the week’s top stories
---------
---------
['need', 'know', 'week', 'stories']
['Here', '’s', 'what', 'you', 'to', 'about', 'the', '’s', 'top']
---------
---------
The 2016 presidential matchup has accelerated trends that were expected to unfold gradually
---------
---------
['2016', 'presidential', 'matchup', 'accelerated', 'trends', 'expected', 'unfold', 'gradually']
['The', 'has', 'that', 'were', 'to']
---------
---------
Most Americans say they are interested in the issue, but it came up only once in the first two debates and is not among topics listed for the third
---------
---------
['Americans', 'interested', 'issue', ',', 'came', 'deba

---------
Over the course of the campaign, both candidates — though Donald J Trump more frequently than Hillary Clinton — have made statements that are in tension with the Constitution
---------
---------
['course', 'campaign', ',', 'candidates', '—', 'Donald', 'J', 'Trump', 'frequently', 'Hillary', 'Clinton', '—', 'statements', 'tension', 'Constitution']
['Over', 'the', 'of', 'the', 'both', 'though', 'more', 'than', 'have', 'made', 'that', 'are', 'in', 'with', 'the']
---------
---------
Mr Trump showed more discipline than in the first two debates, but a few stumbles left him far short of the decisive victory he needed to shift voters’ perceptions
---------
---------
['Mr', 'Trump', 'showed', 'discipline', 'debates', ',', 'stumbles', 'left', 'far', 'short', 'decisive', 'victory', 'needed', 'shift', 'voters', '’', 'perceptions']
['more', 'than', 'in', 'the', 'first', 'two', 'but', 'a', 'few', 'him', 'of', 'the', 'he', 'to']
---------
---------
He’s desperate for a reason to explain why

---------
The candidate’s recent comments about the legitimacy of the election have perversely made some immigrants feel right at home
---------
---------
['candidate', 'recent', 'comments', 'legitimacy', 'election', 'perversely', 'immigrants', 'feel', 'right', 'home']
['The', '’s', 'about', 'the', 'of', 'the', 'have', 'made', 'some', 'at']
---------
---------
If Mrs Clinton had “leaked” her own speeches, few would be talking about them now
---------
---------
['Mrs', 'Clinton', '“', 'leaked', '”', 'speeches', ',', 'talking']
['If', 'had', 'her', 'own', 'few', 'would', 'be', 'about', 'them', 'now']
---------
---------
With a strong lead in national polls, Mrs Clinton is urging core Democratic constituencies to vote early in states where balloting has begun
---------
---------
['strong', 'lead', 'national', 'polls', ',', 'Mrs', 'Clinton', 'urging', 'core', 'Democratic', 'constituencies', 'vote', 'early', 'states', 'balloting', 'begun']
['With', 'a', 'in', 'is', 'to', 'in', 'where', 'has

In [53]:
data.head()

,Date,Headline,Snippet,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment,Headline_polarity,Snippet_polarity,Headline_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Snippet_Cleaned,Snippet_Keywords,Snippet_Stopwords,Snippet_Tokens
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,-0.4767,0,Donald Trump Opens New Line of Attack on Hilla...,"Donald, Trump, Opens, New, Line, Attack, Hilla...","of, on, Her","Donald, Trump, Opens, New, Line, of, Attack, o...","In an interview, Mr Trump said he was reconsid...","interview, ,, Mr, Trump, said, reconsidering, ...","In, an, he, was, whether, he, would, back, if,...","In, an, interview, ,, Mr, Trump, said, he, was..."
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'com...",neu,0,0.5859,Hillary Clinton and Democrats Raised $154 Mill...,"Hillary, Clinton, Democrats, Raised, $, 154, M...","and, in","Hillary, Clinton, and, Democrats, Raised, $, 1...",Her campaign and the party have a combined cas...,"campaign, party, combined, cash, reserve, $, 1...","Her, and, the, have, a, of, them, up, for, a, ...","Her, campaign, and, the, party, have, a, combi..."
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...,"[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'com...",neu,"[{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'com...",neu,0.3182,0.2732,New Hillary Clinton Ads Look to Expand Lead Wi...,"New, Hillary, Clinton, Ads, Look, Expand, Lead...","to, With","New, Hillary, Clinton, Ads, Look, to, Expand, ...",The Clinton team has made Hispanic outreach a ...,"Clinton, team, Hispanic, outreach, cornerstone...","The, has, made, a, of, its, and, are, In, a, h...","The, Clinton, team, has, made, Hispanic, outre..."
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0,How Could Anyone Vote for Trump?,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?",It’s time for Republican leaders to exercise a...,"time, Republican, leaders, exercise, little, l...","It, ’s, for, to, a","It, ’s, time, for, Republican, leaders, to, ex..."
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...,"[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu,"[{'neg': 0.116, 'neu': 0.884, 'pos': 0.0, 'com...",neu,0.296,-0.4404,Donald Trump Is Seen as Helping Push Asian-Ame...,"Donald, Trump, Seen, Helping, Push, Asian, -, ...","Is, as, Into","Donald, Trump, Is, Seen, as, Helping, Push, As...",Asian-Americans are identifying as Democrats a...,"Asian, -, Americans, identifying, Democrats, q...","are, as, at, a, than, any, other, and, many, t...","Asian, -, Americans, are, identifying, as, Dem..."


In [48]:
data.drop(columns=['Keywords','Stopwords','Tokens'], inplace = True)

# Sentiment

#### "VADER has been found to be quite successful when dealing with social media texts, NY Times editorials, movie reviews, and product reviews. "

With VADER no need for removing stop words as its built-in and punctuation is part of the analysis

In [13]:
import spacy
from spacy.tokens import Doc
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlp = spacy.load('en_core_web_sm') 
sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
 
Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Sentiment_Values'] = ""
data['Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Sentiment_Values'] = senti   
    data.at[i,'Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

---------
Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage
---------
---------
[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compound': -0.4767}]
---------
---------
Hillary Clinton and Democrats Raised $154 Million in September
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
New Hillary Clinton Ads Look to Expand Lead With Hispanic Voters
---------
---------
[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.3182}]
---------
---------
How Could Anyone Vote for Trump?
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Donald Trump Is Seen as Helping Push Asian-Americans Into Democratic Arms
---------
---------
[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compound': 0.296}]
---------
---------
Donald Trump’s Pathetic Fraternity
---------
---------
[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'compound': -0.5719}]
---------
---------
On the Trail: Week of Sept. 25
---------
-------

---------
Donald Trump Adjusts His Complaints About Debate Microphone
---------
---------
[{'neg': 0.278, 'neu': 0.722, 'pos': 0.0, 'compound': -0.4019}]
---------
---------
When Trump vs. Clinton Becomes a Family Affair
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
U.S. Says Russia Directed Hacks to Influence Elections
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Democrats Push to Extend Voter Registration in Storm-Damaged Florida
---------
---------
[{'neg': 0.0, 'neu': 0.825, 'pos': 0.175, 'compound': 0.1779}]
---------
---------
Hurricane Matthew, Donald Trump, Nobel Prize: Your Friday Evening Briefing
---------
---------
[{'neg': 0.0, 'neu': 0.732, 'pos': 0.268, 'compound': 0.5106}]
---------
---------
To Redefine Homestretch, Hillary Clinton Cues the Children
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
John McCain Withdraws Suppo

---------
What This 2012 Map Tells Us About America, and the Election
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Teaching Seventh Graders in a ‘Total Mess’ of an Election Season
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Obama Tells Trump: Stop ‘Whining’ and Trying to Discredit the Election
---------
---------
[{'neg': 0.18, 'neu': 0.82, 'pos': 0.0, 'compound': -0.296}]
---------
---------
Pro-Clinton ‘Super PAC’ to Tie Embattled Senators to Donald Trump in Ads
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
70 Nobel Laureates Endorse Hillary Clinton
---------
---------
[{'neg': 0.0, 'neu': 0.685, 'pos': 0.315, 'compound': 0.3182}]
---------
---------
Pledging the Trump Fraternity
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Shameful Silence on Donald Trump’s Lies About Vote-Rigging
-

In [17]:
#most headlines seem to fall under neutral, maybe better to paint directly with the polarity metric

data.head(10)

,Date,Headline,Snippet,Sentiment_Values,Sentiment
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...,"[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'com...",neu
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...,"[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu
5,2016-10-01,Donald Trump’s Pathetic Fraternity,Chris Christie and Rudy Giuliani once had prid...,"[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg
6,2016-10-01,On the Trail: Week of Sept. 25,Hillary Clinton and Donald J. Trump traded bar...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
7,2016-10-01,Trump and the Intellectuals,A case for Trump’s ideas that ignores the man ...,"[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu
8,2016-10-01,The Monster Sorority of Women Voters,The danger of focusing on the differences betw...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
9,2016-10-02,‘S.N.L.’ Begins a New Season With Alec Baldwin...,Kate McKinnon returned as Hillary Clinton in t...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu


In [20]:
#let's try analysing the snippets as well

Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Snippet_Sentiment_Values'] = ""
data['Snippet_Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Snippet_Sentiment_Values'] = senti   
    data.at[i,'Snippet_Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

---------
In an interview, Mr. Trump said he was reconsidering whether he would back Mrs. Clinton if she became president.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Her campaign and the party have a combined cash reserve of $150 million, setting them up for a final spending splurge on television ads and outreach.
---------
---------
[{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'compound': 0.5859}]
---------
---------
The Clinton team has made Hispanic outreach a cornerstone of its advertising, and results are showing: In Florida, a poll shows her as having a 35-point lead.
---------
---------
[{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'compound': 0.2732}]
---------
---------
It’s time for Republican leaders to exercise a little leadership.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Asian-Americans are identifying as Democrats at a quicker pace than any other racial group, and many Rep

---------
His Donald Trump bore little resemblance to reality.
---------
---------
[{'neg': 0.222, 'neu': 0.778, 'pos': 0.0, 'compound': -0.25}]
---------
---------
45Committee is connected to the family of Todd Ricketts, a Republican donor who was once part of the so-called Never Trump movement.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
The popular first lady, who limits her campaign appearances, is being dispatched to places and constituencies where the campaign is facing the greatest pressure.
---------
---------
[{'neg': 0.073, 'neu': 0.695, 'pos': 0.232, 'compound': 0.7003}]
---------
---------
Asked during a number of interviews on Thursday about why he will not condemn the Muslim ban now, Mr. Pence said, “Because that’s not Donald Trump’s position now.”
---------
---------
[{'neg': 0.116, 'neu': 0.772, 'pos': 0.112, 'compound': -0.2769}]
---------
---------
Mr. Trump has fomented divisions that began before he started his cam

---------
Follow along, and discuss, the second presidential debate with our columnists Frank Bruni, Roger Cohen, Gail Collins, and Ross Douthat.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Many people were fascinated by Mr. Bone, the undecided voter in the red sweater who asked a policy question Sunday night.
---------
---------
[{'neg': 0.083, 'neu': 0.783, 'pos': 0.135, 'compound': 0.296}]
---------
---------
Mr. Trump, offering no evidence, suggested to a crowd in New Hampshire that Hillary Clinton may have been on drugs during their last debate.
---------
---------
[{'neg': 0.091, 'neu': 0.909, 'pos': 0.0, 'compound': -0.296}]
---------
---------
Mrs. Clinton has been scarred by old missteps and paralyzed by her husband’s history of facing accusations of sexual misconduct.
---------
---------
[{'neg': 0.108, 'neu': 0.892, 'pos': 0.0, 'compound': -0.3182}]
---------
---------
The public’s concern about the abuses of big money in e

---------
As polls show Donald J. Trump losing in must-win states, he and Hillary Clinton debated one last time.
---------
---------
[{'neg': 0.133, 'neu': 0.867, 'pos': 0.0, 'compound': -0.3818}]
---------
---------
In response to a Trump campaign plan at the last debate to parade women who accused Mr. Clinton of sexual assault into the hall, the Clinton campaign has gained approval to change entry protocol.
---------
---------
[{'neg': 0.147, 'neu': 0.713, 'pos': 0.14, 'compound': -0.0772}]
---------
---------
Presidential rivals typically run roughly the same number of commercials. Donald Trump’s lack of advertising seems to be costing him.
---------
---------
[{'neg': 0.106, 'neu': 0.833, 'pos': 0.06, 'compound': -0.25}]
---------
---------
Lt. Gen. Michael T. Flynn appears likely to emerge as the angry voice of what could best be described as the alternative right of the American national security establishment.
---------
---------
[{'neg': 0.092, 'neu': 0.724, 'pos': 0.184, 'comp

---------
Republicans control most statehouse chambers, but with Donald J. Trump stumbling, Democrats hope for down-ballot gains in Minnesota, Colorado, Nevada and elsewhere.
---------
---------
[{'neg': 0.0, 'neu': 0.742, 'pos': 0.258, 'compound': 0.7876}]
---------
---------
In June in San Jose, Calif., a tug of war over a sign supporting Donald J. Trump sent one man to jail for six days and another man to soul-searching. This is their story.
---------
---------
[{'neg': 0.106, 'neu': 0.815, 'pos': 0.079, 'compound': -0.25}]
---------
---------
Big deals are usually postponed during times of political and economic uncertainty, but many corporate titans apparently see smooth sailing ahead.
---------
---------
[{'neg': 0.14, 'neu': 0.86, 'pos': 0.0, 'compound': -0.2732}]
---------
---------
Here’s what you need to know to start your day.
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
The state has voted for the G.O.P. candidate in the pa

In [24]:
data.head(100)

,Date,Headline,Snippet,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'com...",neu
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...,"[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'com...",neu,"[{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'com...",neu
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...,"[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu,"[{'neg': 0.116, 'neu': 0.884, 'pos': 0.0, 'com...",neu
5,2016-10-01,Donald Trump’s Pathetic Fraternity,Chris Christie and Rudy Giuliani once had prid...,"[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg,"[{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'com...",neu
6,2016-10-01,On the Trail: Week of Sept. 25,Hillary Clinton and Donald J. Trump traded bar...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
7,2016-10-01,Trump and the Intellectuals,A case for Trump’s ideas that ignores the man ...,"[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu,"[{'neg': 0.208, 'neu': 0.792, 'pos': 0.0, 'com...",neu
8,2016-10-01,The Monster Sorority of Women Voters,The danger of focusing on the differences betw...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'com...",neu
9,2016-10-02,‘S.N.L.’ Begins a New Season With Alec Baldwin...,Kate McKinnon returned as Hillary Clinton in t...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu


In [31]:
data['Sentiment_Values'][2][0]['compound']

0.3182

In [32]:
data.Headline.iloc[0]

'Donald Trump Opens New Line of Attack on Hillary Clinton: Her Marriage'

In [34]:
data['Headline_polarity'] = data['Headline']

for i in range(len(data.Sentiment)):
    data['Headline_polarity'].iloc[i] = data['Sentiment_Values'][i][0]['compound']

In [36]:
data['Snippet_polarity'] = data['Headline']

for i in range(len(data.Sentiment)):
    data['Snippet_polarity'].iloc[i] = data['Snippet_Sentiment_Values'][i][0]['compound']

In [38]:
data.head(30)

,Date,Headline,Snippet,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment,Headline_polarity,Snippet_polarity
0,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,-0.4767,0
1,2016-10-01,Hillary Clinton and Democrats Raised $154 Mill...,Her campaign and the party have a combined cas...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 0.821, 'pos': 0.179, 'com...",neu,0,0.5859
2,2016-10-01,New Hillary Clinton Ads Look to Expand Lead Wi...,The Clinton team has made Hispanic outreach a ...,"[{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'com...",neu,"[{'neg': 0.0, 'neu': 0.916, 'pos': 0.084, 'com...",neu,0.3182,0.2732
3,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0
4,2016-10-01,Donald Trump Is Seen as Helping Push Asian-Ame...,Asian-Americans are identifying as Democrats a...,"[{'neg': 0.0, 'neu': 0.82, 'pos': 0.18, 'compo...",neu,"[{'neg': 0.116, 'neu': 0.884, 'pos': 0.0, 'com...",neu,0.296,-0.4404
5,2016-10-01,Donald Trump’s Pathetic Fraternity,Chris Christie and Rudy Giuliani once had prid...,"[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg,"[{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'com...",neu,-0.5719,0.34
6,2016-10-01,On the Trail: Week of Sept. 25,Hillary Clinton and Donald J. Trump traded bar...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0
7,2016-10-01,Trump and the Intellectuals,A case for Trump’s ideas that ignores the man ...,"[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu,"[{'neg': 0.208, 'neu': 0.792, 'pos': 0.0, 'com...",neu,0.3818,-0.2732
8,2016-10-01,The Monster Sorority of Women Voters,The danger of focusing on the differences betw...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.152, 'neu': 0.848, 'pos': 0.0, 'com...",neu,0,-0.5267
9,2016-10-02,‘S.N.L.’ Begins a New Season With Alec Baldwin...,Kate McKinnon returned as Hillary Clinton in t...,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0


# Frequency

In [40]:
#for frquency need to tokenize
#to tokenize need to clear punctuations

from collections import Counter

def wordextractdf(df, col):
    """
    Extract words in the dataframe

    Inputs:
    df -- dataframe
    col -- set of words that we can extract ('keywords', 'hashtags',...)

    Outputs:
    wordsfreq - Counter containing a list of words and its frequency in the dataframe
    """
    words = df[col][df[col].isnull() == False]
    lst = [w for word in words for w in word.split(",")]
    return Counter(lst)

def calc_freq(df, col):
    kw = wordextractdf(df, col)
    dictlist = [[key, value] for key, value in kw.items()]
    freq_df = pd.DataFrame()
    freq_df['words'] = [i[0] for i in dictlist]
    freq_df['freq_total'] = [i[1] for i in dictlist]
    return freq_df

sorted(wordextractdf(data, 'Headline').most_common())
calc_freq(data, 'Headline')

,words,freq_total
0,Donald Trump Opens New Line of Attack on Hilla...,1
1,Hillary Clinton and Democrats Raised $154 Mill...,1
2,New Hillary Clinton Ads Look to Expand Lead Wi...,1
3,How Could Anyone Vote for Trump?,1
4,Donald Trump Is Seen as Helping Push Asian-Ame...,1
5,Donald Trump’s Pathetic Fraternity,1
6,On the Trail: Week of Sept. 25,1
7,Trump and the Intellectuals,1
8,The Monster Sorority of Women Voters,1
9,‘S.N.L.’ Begins a New Season With Alec Baldwin...,1


# Export

In [54]:
data.to_csv('data/nyt_sentiment.csv', index=False)